In [1]:
import copy
import datetime
import pandas as pd
import scrapbook as sb
from dateutil import tz

import systemlink.clients.nitestmonitor as testmon

In [2]:
results_api = testmon.ResultsApi()

In [3]:
results_filter = 'startedWithin <= "1.0:0:0"'
products_filter = ''

In [4]:
results_query = testmon.ResultsAdvancedQuery(
    results_filter, product_filter=products_filter, order_by=testmon.ResultField.STARTED_AT)

results = []

response = await results_api.query_results_v2(post_body=results_query)
while response.continuation_token:
    results = results + response.results
    results_query.continuation_token = response.continuation_token
    response = await results_api.query_results_v2(post_body=results_query)

results_list = [result.to_dict() for result in results]

In [5]:
formatted_results = {
    'status': [result['status']['status_type'] for result in results_list],
    'started_at': [result['started_at'] for result in results_list],
    'location': [result['properties']['Location'] for result in results_list]
}

In [6]:
df = pd.DataFrame.from_dict(formatted_results)

In [7]:
df

,status,started_at,location
0,FAILED,2022-02-20 20:37:07.082217+00:00,EPL-AR1-Line 1
1,PASSED,2022-02-20 20:37:09.972302+00:00,EPL-AR1-Line 1
2,PASSED,2022-02-20 20:37:14.651153+00:00,EPL-AR1-Line 1
3,PASSED,2022-02-20 20:37:16.510815+00:00,EPL-AR1-Line 1
4,PASSED,2022-02-20 20:37:22.849411+00:00,EPL-AR1-Line 1
...,...,...,...
20237,PASSED,2022-02-21 03:51:12.067413+00:00,EPL-AR2-Line 3
20238,PASSED,2022-02-21 03:51:24.121739+00:00,EPL-AR2-Line 3
20239,RUNNING,2022-02-21 03:51:27.042493+00:00,EPL-AR2-Line 3
20240,RUNNING,2022-02-21 03:51:33.564253+00:00,EPL-AR2-Line 3


In [8]:
filt = df['location'].str.contains('EPL')

In [9]:
df = df.loc[filt]

In [10]:
df = df.groupby(['location', 'status']).count()

In [11]:
df

started_at
location       status             
EPL-AR1-Line 1 FAILED           93
               PASSED         2130
               RUNNING           3
EPL-AR1-Line 2 FAILED          111
               PASSED         2226
               RUNNING           4
EPL-AR1-Line 5 FAILED           45
               PASSED         1347
               RUNNING           1
EPL-AR1-Line 6 FAILED           16
               PASSED          675
               RUNNING           4
EPL-AR2-Line 1 FAILED          202
               PASSED         3204
               RUNNING           2
EPL-AR2-Line 2 FAILED          183
               PASSED         3148
               RUNNING           3
EPL-AR2-Line 3 FAILED          105
               PASSED         3462
               RUNNING           3
EPL-AR2-Line 4 FAILED           19
               PASSED         1184
               RUNNING           3
EPL-AR2-Line 5 FAILED           97
               PASSED         1942

In [12]:
df = df.unstack()

In [13]:
df

started_at                
status             FAILED  PASSED RUNNING
location                                 
EPL-AR1-Line 1       93.0  2130.0     3.0
EPL-AR1-Line 2      111.0  2226.0     4.0
EPL-AR1-Line 5       45.0  1347.0     1.0
EPL-AR1-Line 6       16.0   675.0     4.0
EPL-AR2-Line 1      202.0  3204.0     2.0
EPL-AR2-Line 2      183.0  3148.0     3.0
EPL-AR2-Line 3      105.0  3462.0     3.0
EPL-AR2-Line 4       19.0  1184.0     3.0
EPL-AR2-Line 5       97.0  1942.0     NaN

In [14]:
columns = []

In [15]:
for column in df.columns:
    columns.append(column[1].lower())

In [16]:
columns

['failed', 'passed', 'running']

In [17]:
df.columns = columns

In [18]:
df.fillna(0, inplace=True)

In [19]:
total_tested = 0

In [20]:
for column in columns:
    total_tested += df[column]

In [21]:
df['pass_rate'] = df['passed'] / (df['passed'] + df['failed'] + df['running'] * 100)

In [22]:
df.reset_index(inplace=True)

In [23]:
df_dict = {
    'columns': pd.io.json.build_table_schema(df, index=False)['fields'],
    'values': df.values.tolist(),
}

pass_rate = {
    'type': 'data_frame',
    'id': 'results_graph',
    'data': df_dict
}

In [24]:
result = [pass_rate]

In [25]:
sb.glue('result', result)

In [26]:
result

[{'type': 'data_frame',
  'data': {'columns': [{'name': 'location', 'type': 'string'},
    {'name': 'failed', 'type': 'number'},
    {'name': 'passed', 'type': 'number'},
    {'name': 'running', 'type': 'number'},
    {'name': 'pass_rate', 'type': 'number'}],
   'values': [['EPL-AR1-Line 1', 93.0, 2130.0, 3.0, 0.8442330558858502],
    ['EPL-AR1-Line 2', 111.0, 2226.0, 4.0, 0.8132992327365729],
    ['EPL-AR1-Line 5', 45.0, 1347.0, 1.0, 0.9028150134048257],
    ['EPL-AR1-Line 6', 16.0, 675.0, 4.0, 0.618698441796517],
    ['EPL-AR2-Line 1', 202.0, 3204.0, 2.0, 0.8885191347753744],
    ['EPL-AR2-Line 2', 183.0, 3148.0, 3.0, 0.866978793720738],
    ['EPL-AR2-Line 3', 105.0, 3462.0, 3.0, 0.8952676493405741],
    ['EPL-AR2-Line 4', 19.0, 1184.0, 3.0, 0.7877578176979375],
    ['EPL-AR2-Line 5', 97.0, 1942.0, 0.0, 0.9524276606179499]]}}]